# 数据预处理

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('shanghai_fishing_data.csv').dropna().reset_index()
data['lng'] = np.nan
data['lat'] = np.nan

In [3]:
len(data)

880

In [4]:
data.iloc[0]

index                                                          1
标题                                                           太浦河
标题链接           https://www.haodiaoyu.com/diaochang/shanghai/5...
缩略图            http://p6.diaoyur.cn/group6/M00/06/ED/cjd0iVX0...
right-info                                       上海市青浦区练西公路2755弄
right-info1                                                   免费
lng                                                          NaN
lat                                                          NaN
Name: 0, dtype: object

# 通过地址获得经纬度

In [5]:
import json 
from urllib.request import urlopen, quote
def get_location(address):
    url = 'http://api.map.baidu.com/geocoding/v3/'
    output = 'json'
    ak = 'Z2DhjlINisiQofu9MRGzPBhsjabYOTon'
    add = quote(address)
    uri = url + '?' + 'address=' + add + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    try:
        return temp['result']
    except:
        # print('\n'+str(temp))
        return ''
get_location('上海市浦东新区运航路909号')

{'location': {'lng': 121.77481532542775, 'lat': 31.148866824133297},
 'precise': 1,
 'confidence': 80,
 'comprehension': 100,
 'level': '门址'}

# 给每一条数据加上经纬度

In [6]:
from tqdm import tqdm
import time
import numpy as np
n = len(data)
lng = [0]*n
lat = [0]*n
failure = []
for i in tqdm(range(n)):
    location = data.iloc[i][4]
    try:
        information = get_location(location)
        data.loc[i,'lng'] = information['location']['lng']
        data.loc[i,'lat'] = information['location']['lat']
    except:
        failure.append(i)

100%|██████████| 880/880 [00:53<00:00, 16.53it/s]


In [10]:
new_data = data.dropna().reset_index()

In [13]:
new_data.iloc[0][6]

'免费'

# 利用百度地图绘制钓点

In [14]:
from pyecharts.charts import Geo
from pyecharts import options as opts
from pyecharts.globals import GeoType
from pyecharts.charts import BMap
import json as js
def convert_data(data):
    res = []
    for i in range(len(data)):
        res.append([data.iloc[i][2], data.iloc[i][6]])
    return res
data_pair = convert_data(new_data)

In [15]:
m = BMap(init_opts=opts.InitOpts(width="1400px", height="800px")).add_schema(
        baidu_ak="Z2DhjlINisiQofu9MRGzPBhsjabYOTon",\
        center=[121.47,31.17],\
        zoom=11,\
        map_style=js.load(open('mapstyle.json','r'))\
    )
for i in range(len(new_data)):
    m.add_coordinate(new_data.iloc[i,2],new_data.iloc[i,7],new_data.iloc[i,8])
m = m.add(
        type_="effectScatter",
        series_name="fishing_point",
        data_pair=data_pair,
        symbol_size=10,
        effect_opts=opts.EffectOpts(),
        label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=False),
        itemstyle_opts=opts.ItemStyleOpts(color="red"),
    )
m.render("fishing_shanghai_map.html")

'/Users/luoruipu/Fishing_Map_Shanghai_Visualization/fishing_shanghai_map.html'

In [16]:
new_data.to_csv('new_data.csv',index=False)